In [21]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas

In [22]:
# CHOOSE DATASET

# Regression dataset
data = datasets.load_diabetes(as_frame=True)

# Classification dataset
#data = datasets.fetch_openml("diabetes",version=1, as_frame=True) 

X = data.data.values
y = data.target.values 
X.shape

(442, 10)

In [23]:
#train test spliting
test_size=0.2
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=42)

In [24]:
# Standardize features
scaler=StandardScaler()
Xtr= scaler.fit_transform(Xtr)
Xte= scaler.transform(Xte)

In [25]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size=1, dropout_prob=0.5):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.out = nn.Linear(128, output_size)
        
        self.dropout = nn.Dropout(p=dropout_prob)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        
        x = self.out(x)
        return x

In [26]:
num_epochs=500
lr=0.00025
dropout=0.05
batch_size=64

In [27]:
Xtr = torch.tensor(Xtr, dtype=torch.float32)
ytr = torch.tensor(ytr, dtype=torch.float32)
Xte = torch.tensor(Xte, dtype=torch.float32)
yte = torch.tensor(yte, dtype=torch.float32)

# Wrap Xtr and ytr into a dataset
train_dataset = TensorDataset(Xtr, ytr)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [28]:
# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(input_size=Xtr.shape[1], dropout_prob=dropout).to(device)
#criterion = nn.BCEWithLogitsLoss()  # for binary classification
criterion = nn.MSELoss() #for regression
optimizer = optim.Adam(model.parameters(), lr=lr)

In [29]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for batch_x, batch_y in train_dataloader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        logits = model(batch_x)
        loss = criterion(logits, batch_y.view(-1, 1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/500], Loss: 29438.4925
Epoch [2/500], Loss: 29268.9616
Epoch [3/500], Loss: 29619.7917
Epoch [4/500], Loss: 29000.3421
Epoch [5/500], Loss: 29456.8353
Epoch [6/500], Loss: 29823.8522
Epoch [7/500], Loss: 29636.6051
Epoch [8/500], Loss: 29275.9593
Epoch [9/500], Loss: 29357.8249
Epoch [10/500], Loss: 28433.1009
Epoch [11/500], Loss: 26363.3385
Epoch [12/500], Loss: 26054.6383
Epoch [13/500], Loss: 24292.2386
Epoch [14/500], Loss: 21670.8688
Epoch [15/500], Loss: 18251.5378
Epoch [16/500], Loss: 14319.1774
Epoch [17/500], Loss: 11059.0828
Epoch [18/500], Loss: 8366.9289
Epoch [19/500], Loss: 6435.3175
Epoch [20/500], Loss: 5556.5446
Epoch [21/500], Loss: 5172.9698
Epoch [22/500], Loss: 4834.4103
Epoch [23/500], Loss: 4475.1899
Epoch [24/500], Loss: 4315.3626
Epoch [25/500], Loss: 4441.7754
Epoch [26/500], Loss: 4152.5026
Epoch [27/500], Loss: 3959.4728
Epoch [28/500], Loss: 3958.5800
Epoch [29/500], Loss: 3798.0827
Epoch [30/500], Loss: 3810.9907
Epoch [31/500], Loss: 4009.1216


In [30]:
model.eval()
y_pred=model(Xte)
#print(f'ACC:{accuracy_score(yte.detach().numpy(),y_pred.detach().numpy()>0.5)}') #classification
print(f'MSE:{mean_squared_error(yte.detach().numpy(),y_pred.detach().numpy())}') #regression

MSE:2760.140869140625
